# Webexport
!! tippecanoe installieren !!

```
brew install tippecanoe
```

In [5]:
import geopandas as gpd
import pandas as pd
import json
import os
import consts
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time

In [6]:
# Load timedata
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

# Reduce precision
#gdf_grid_raw['geometry'] = gdf_grid_raw['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

In [7]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'name': x['name'],
    'sunrise_current_dt': x['sunrise']['current']['latest'],
    'sunrise_summertime': x['sunrise']['summer']['latest'],
    'sunrise_wintertime': x['sunrise']['winter']['latest']
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['sunrise_current'] = df_timedata['sunrise_current_dt'].apply(dt_to_time)
df_timedata['sunrise_summertime'] = df_timedata['sunrise_summertime'].apply(time_to_time)
df_timedata['sunrise_wintertime'] = df_timedata['sunrise_wintertime'].apply(time_to_time)

# Current System
df_timedata['sunrise_current_category'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_current_numerical'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Summertime
df_timedata['sunrise_summertime_category'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_summertime_numerical'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Wintertime
df_timedata['sunrise_wintertime_category'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_wintertime_numerical'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

In [9]:
df_timedata['sunrise_summertime_numerical'].max()

np.float64(13.0)

## Create fill-color-Expression for Maptiler

In [23]:
field = 'sunrise_summertime_numerical'
colors = ["#93144b", "#a02c49", "#ad3e4a", "#b74f4b", "#c1604e", "#c96f52", "#ce7f58", "#d38f62", "#d59e6d", "#d8ad7c", "#dabb8c", "#dcc79e", "#e0d4b1"]

stepper = 0
values = []
for i in range(int(df_timedata[field].min() * 2), int(df_timedata[field].max() * 2) + 1):
    value = i / 2
    values.append(value)
    values.append(f'"{colors[stepper]}"')
    stepper += 1


print("""
 [
      "interpolate",
      [
        "linear"
      ],
      [
        "get",
        "sunrise_summertime_numerical"
      ],
      """ + '\n      ,'.join(map(str, values)) + """
    ],""")


 [
      "interpolate",
      [
        "linear"
      ],
      [
        "get",
        "sunrise_summertime_numerical"
      ],
      7.5
      ,"#93144b"
      ,8.0
      ,"#a02c49"
      ,8.5
      ,"#ad3e4a"
      ,9.0
      ,"#b74f4b"
      ,9.5
      ,"#c1604e"
      ,10.0
      ,"#c96f52"
      ,10.5
      ,"#ce7f58"
      ,11.0
      ,"#d38f62"
      ,11.5
      ,"#d59e6d"
      ,12.0
      ,"#d8ad7c"
      ,12.5
      ,"#dabb8c"
      ,13.0
      ,"#dcc79e"
    ],


## Export GeoJSON

In [11]:
gdf_grid.to_file(consts.PATH_GEOJSON_EXPORT)

## Convert with Tippecanoe

In [ ]:
# Run Tippecanoe
os.system(f"tippecanoe -zg -o {consts.PATH_MBTILE_EXPORT} --drop-densest-as-needed {consts.PATH_GEOJSON_EXPORT} --force")

For layer 0, using name "grid"
6001 features, 1061212 bytes of geometry, 445878 bytes of string pool
Choosing a maxzoom of -z1 for features typically 161371 feet (49186 meters) apart, and at least 60873 feet (18555 meters) apart
Choosing a maxzoom of -z4 for resolution of about 20210 feet (6160 meters) within features
  99.9%  4/9/6  


0